In [115]:
import pandas as pd
import jax
from jax import numpy as jnp
from jax import random
from flax import linen as nn
from flax.training import train_state
import optax

In [116]:
df = pd.read_pickle("grouped_df.pkl")
df_train = df["2000-01-01":"2017-01-01"]
df_test = df["2017-01-01":"2019-01-01"]

C:\Users\12lui\AppData\Local\Temp\ipykernel_6568\2169528782.py:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df_train = df["2000-01-01":"2017-01-01"]
C:\Users\12lui\AppData\Local\Temp\ipykernel_6568\2169528782.py:3: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df_test = df["2017-01-01":"2019-01-01"]


In [117]:
class BayesianNN(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(1155)(x)
        x = nn.relu(x)
        x = nn.Dense(512)(x)
        x = nn.relu(x)
        x = nn.Dense(1)(x)
        return x

In [118]:
# Define the loss function
def log_sigmoid(x):
    return -jnp.log(1 + jnp.exp(-x))

def binary_cross_entropy(logits, labels):
    logprobs = log_sigmoid(logits) * (1 - labels) + (log_sigmoid(logits) - logits) * labels
    return -jnp.mean(logprobs)

In [119]:
# Define the optimizer
optimizer = optax.adam(learning_rate=0.001)

In [120]:
# Initialize the model
key = random.PRNGKey(0)
initial_params = BayesianNN().init(key, jnp.ones((1, df_train.shape[1]-1)))
state = train_state.TrainState.create(
    apply_fn=BayesianNN().apply,
    params=initial_params,
    tx=optimizer,
)

In [121]:
# Training step
@jax.jit
def train_step(state, batch):
    def loss_fn(params):
        logits = state.apply_fn(params, batch["input"])
        loss = binary_cross_entropy(logits, batch["label"])
        return loss
    grad_fn = jax.value_and_grad(loss_fn)
    loss, grad = grad_fn(state.params)
    state = state.apply_gradients(grads=grad)
    return state, loss

In [122]:
# Initialize the training state
rng = jax.random.PRNGKey(0)
batch_size = 64
num_epochs = 50

In [123]:
# Training loop
for epoch in range(num_epochs):
    for i in range(0, len(df_train), batch_size):
        batch_input = df_train.iloc[i:i + batch_size, :-1].values
        batch_label = df_train.iloc[i:i + batch_size, -1].values
        batch = {"input": batch_input, "label": batch_label}
        state, loss = train_step(state, batch)

In [124]:
def accuracy(params, batch):
    inputs, targets = batch
    logits = state.apply_fn(params, inputs)
    predicted = jnp.round(jax.nn.sigmoid(logits))
    return jnp.mean(predicted == targets)

In [125]:
test_batch

{'input': array([[ 1.1285163 ,  0.58350284,  0.59466237, ...,  0.        ,
          0.        ,  0.        ],
        [-0.21005125,  1.46029109,  0.59114704, ...,  0.        ,
          0.        ,  0.        ],
        [-0.24997031, -0.29574433, -0.2858963 , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.52457258,  1.83987033,  1.25098509, ...,  0.        ,
          0.        ,  0.        ],
        [-0.76535472, -0.53603427, -0.2658714 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.52585318,  0.83832734,  1.00361173, ...,  0.        ,
          0.        ,  0.        ]]),
 'label': array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)}

In [126]:
test_batch_input = df_test.iloc[:, :-1].values
test_batch_label = df_test.iloc[:, -1].values

test_batch = {"input": test_batch_input, "label": test_batch_label}
test_accuracy = accuracy(state.params, test_batch)
print('Test accuracy:', test_accuracy)

IndexError: tuple index out of range